In [ ]:
import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler, Binarizer, OneHotEncoder
from scipy import stats

# Load the dataset
df = pd.read_csv('healthcare_dataset.csv')


In [ ]:
# 1.Calculate percentage of missing values for each column
missing_percentage = df.isnull().mean() * 100
print("Percentage of missing values in each column:")
print(missing_percentage)
print("there is no missing values_in this dataset")

Percentage of missing values in each column:
Name                  0.0
Age                   0.0
Gender                0.0
Blood Type            0.0
Medical Condition     0.0
Date of Admission     0.0
Doctor                0.0
Hospital              0.0
Insurance Provider    0.0
Billing Amount        0.0
Room Number           0.0
Admission Type        0.0
Discharge Date        0.0
Medication            0.0
Test Results          0.0
dtype: float64
there is no missing values_in this dataset



```python
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler, Binarizer, OneHotEncoder

# Load the dataset
file_path = 'healthcare_dataset.csv'
df = pd.read_csv(file_path)

# Display the first few rows of the dataset
print("Original Dataset:")
print(df.head())

# Adding 5 rows with some missing values for experimentation
missing_data = pd.DataFrame({
    'Name': ['John Doe', 'Jane Smith', 'Alice Brown', 'Bob White', 'Eve Black'],
    'Age': [np.nan, 40, 35, np.nan, 55],
    'Gender': ['Male', 'Female', 'Female', 'Male', np.nan],
    'Blood Type': ['A+', np.nan, 'O-', 'B+', 'A-'],
    'Medical Condition': ['Hypertension', 'Cancer', np.nan, 'Asthma', 'Diabetes'],
    'Date of Admission': ['2023-01-01', '2023-01-02', '2023-01-03', '2023-01-04', '2023-01-05'],
    'Doctor': ['Dr. A', 'Dr. B', 'Dr. C', 'Dr. D', 'Dr. E'],
    'Hospital': ['Hospital A', 'Hospital B', 'Hospital C', 'Hospital D', 'Hospital E'],
    'Insurance Provider': ['Provider A', 'Provider B', np.nan, 'Provider D', 'Provider E'],
    'Billing Amount': [5000, np.nan, 7000, 8000, 9000],
    'Room Number': [101, 102, 103, 104, 105],
    'Admission Type': ['Elective', 'Emergency', 'Urgent', np.nan, 'Emergency'],
    'Discharge Date': ['2023-01-10', '2023-01-12', '2023-01-14', '2023-01-16', '2023-01-18'],
    'Medication': ['Med A', 'Med B', 'Med C', 'Med D', 'Med E'],
    'Test Results': ['Normal', 'Abnormal', 'Normal', 'Inconclusive', 'Normal']
})

df = pd.concat([df, missing_data], ignore_index=True)

# Display the dataset with missing values
print("Dataset with Missing Values:")
print(df.tail(10))

# 1. Calculate the % of missing values in a column
missing_percentage = df.isnull().mean() * 100
print("Percentage of Missing Values in Each Column:")
print(missing_percentage)

# 2. Replace missing value with mean if the % of missing value is less than 10%.
for col in df.select_dtypes(include=[np.number]).columns:
    if missing_percentage[col] < 10:
        df[col].fillna(df[col].mean(), inplace=True)

# 3. Perform the mode imputation for a categorical data.
for col in df.select_dtypes(include=[object]).columns:
    if missing_percentage[col] < 10:
        df[col].fillna(df[col].mode()[0], inplace=True)

# 4. Perform a KNN Imputer to estimate the missing values.
imputer = KNNImputer(n_neighbors=2)
df_imputed = pd.DataFrame(imputer.fit_transform(df.select_dtypes(include=[np.number])), columns=df.select_dtypes(include=[np.number]).columns)
df.update(df_imputed)

# 5. Drop the columns with more than 10% missing values and display the size.
df_dropped_cols = df.dropna(axis=1, thresh=len(df) * 0.9)
print("Dataset Size after Dropping Columns with >10% Missing Values:", df_dropped_cols.shape)

# 6. Drop the rows with outlier Z-score value > 3 and display the size.
from scipy.stats import zscore
z_scores = df.select_dtypes(include=[np.number]).apply(zscore)
df_no_outliers = df[(np.abs(z_scores) < 3).all(axis=1)]
print("Dataset Size after Dropping Rows with Outlier Z-score > 3:", df_no_outliers.shape)

# 7. Drop the duplicate rows based on more than 50% of column having same value.
df_no_duplicates = df.drop_duplicates(subset=df.columns[:int(len(df.columns) * 0.5)])
print("Dataset Size after Dropping Duplicate Rows based on >50% Columns:", df_no_duplicates.shape)

# 8. Rescale your data using min-max normalization for a numerical feature.
scaler = MinMaxScaler()
df['Billing Amount'] = scaler.fit_transform(df[['Billing Amount']])
print("Billing Amount after Min-Max Normalization:")
print(df[['Billing Amount']].head())

# 9. Binarize the data by using binarizer class in python
binarizer = Binarizer(threshold=5000)
df['Billing Amount Binarized'] = binarizer.fit_transform(df[['Billing Amount']])
print("Billing Amount after Binarization:")
print(df[['Billing Amount Binarized']].head())

# 10. Perform the one-hot encoding for a categorical feature.
encoder = OneHotEncoder(sparse=False)
encoded_columns = encoder.fit_transform(df[['Gender']])
encoded_df = pd.DataFrame(encoded_columns, columns=encoder.get_feature_names_out(['Gender']))
df = df.join(encoded_df)
print("Dataset after One-Hot Encoding 'Gender':")
print(df.head())
```

### Explanation:
1. **Calculate the % of missing values in a column**:
   - This calculates the percentage of missing values in each column.
2. **Replace missing value with mean if the % of missing value is less than 10%**:
   - This replaces missing numerical values with the mean of the column if the percentage of missing values is less than 10%.
3. **Perform the mode imputation for a categorical data**:
   - This replaces missing categorical values with the mode (most frequent value) of the column if the percentage of missing values is less than 10%.
4. **Perform a KNN Imputer to estimate the missing values**:
   - This uses KNN Imputer to estimate and fill missing numerical values.
5. **Drop the columns with more than 10% missing values and display the size**:
   - This drops columns with more than 10% missing values and displays the new dataset size.
6. **Drop the rows with outlier Z-score value > 3 and display the size**:
   - This removes rows that have outlier Z-scores greater than 3 and displays the new dataset size.
7. **Drop the duplicate rows based on more than 50% of column having same value**:
   - This removes duplicate rows based on more than 50% of columns having the same value and displays the new dataset size.
8. **Rescale your data using min-max normalization for a numerical feature**:
   - This normalizes the 'Billing Amount' column to a range between 0 and 1.
9. **Binarize the data by using binarizer class in python**:
   - This binarizes the 'Billing Amount' column with a threshold of 5000.
10. **Perform the one-hot encoding for a categorical feature**:
    - This performs one-hot encoding on the 'Gender' column and adds the new encoded columns to the dataset.

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler, Binarizer, OneHotEncoder

# Load the dataset
file_path = 'healthcare_dataset.csv'
df = pd.read_csv(file_path)

# Display the first few rows of the dataset
print("Original Dataset:")
print(df.head())

# Adding 5 rows with some missing values for experimentation
missing_data = pd.DataFrame({
    'Name': ['John Doe', 'Jane Smith', 'Alice Brown', 'Bob White', 'Eve Black'],
    'Age': [np.nan, 40, 35, np.nan, 55],
    'Gender': ['Male', 'Female', 'Female', 'Male', np.nan],
    'Blood Type': ['A+', np.nan, 'O-', 'B+', 'A-'],
    'Medical Condition': ['Hypertension', 'Cancer', np.nan, 'Asthma', 'Diabetes'],
    'Date of Admission': ['2023-01-01', '2023-01-02', '2023-01-03', '2023-01-04', '2023-01-05'],
    'Doctor': ['Dr. A', 'Dr. B', 'Dr. C', 'Dr. D', 'Dr. E'],
    'Hospital': ['Hospital A', 'Hospital B', 'Hospital C', 'Hospital D', 'Hospital E'],
    'Insurance Provider': ['Provider A', 'Provider B', np.nan, 'Provider D', 'Provider E'],
    'Billing Amount': [5000, np.nan, 7000, 8000, 9000],
    'Room Number': [101, 102, 103, 104, 105],
    'Admission Type': ['Elective', 'Emergency', 'Urgent', np.nan, 'Emergency'],
    'Discharge Date': ['2023-01-10', '2023-01-12', '2023-01-14', '2023-01-16', '2023-01-18'],
    'Medication': ['Med A', 'Med B', 'Med C', 'Med D', 'Med E'],
    'Test Results': ['Normal', 'Abnormal', 'Normal', 'Inconclusive', 'Normal']
})

df = pd.concat([df, missing_data], ignore_index=True)

# Display the dataset with missing values
print("Dataset with Missing Values:")
df.tail(10)


Original Dataset:
                  Name  Age  Gender Blood Type Medical Condition  \
0      Tiffany Ramirez   81  Female         O-          Diabetes   
1          Ruben Burns   35    Male         O+            Asthma   
2            Chad Byrd   61    Male         B-           Obesity   
3    Antonio Frederick   49    Male         B-            Asthma   
4  Mrs. Brandy Flowers   51    Male         O-         Arthritis   

  Date of Admission          Doctor                   Hospital  \
0        2022-11-17  Patrick Parker           Wallace-Hamilton   
1        2023-06-01   Diane Jackson  Burke, Griffin and Cooper   
2        2019-01-09      Paul Baker                 Walton LLC   
3        2020-05-02  Brian Chandler                 Garcia Ltd   
4        2021-07-09  Dustin Griffin    Jones, Brown and Murray   

  Insurance Provider  Billing Amount  Room Number Admission Type  \
0           Medicare    37490.983364          146       Elective   
1   UnitedHealthcare    47304.064845    

In [ ]:
# 1. Calculate the % of missing values in a column
missing_percentage = df.isnull().mean() * 100
print("Percentage of Missing Values in Each Column:")
print(missing_percentage)


Percentage of Missing Values in Each Column:
Name                  0.000000
Age                   0.019990
Gender                0.009995
Blood Type            0.009995
Medical Condition     0.009995
Date of Admission     0.000000
Doctor                0.000000
Hospital              0.000000
Insurance Provider    0.009995
Billing Amount        0.009995
Room Number           0.000000
Admission Type        0.009995
Discharge Date        0.000000
Medication            0.000000
Test Results          0.000000
dtype: float64


In [ ]:
# 2. Replace missing value with mean if the % of missing value is less than 10%.
for col in df.select_dtypes(include=[np.number]).columns:
    if missing_percentage[col] < 10:
        df[col].fillna(df[col].mean(), inplace=True)

In [ ]:
print(f"Missing values in '{col}' filled with mean.") # Print output for this step
print("\nDataset after Replacing Missing Numerical Values with Mean:")
print(df.tail(10))

Missing values in 'Room Number' filled with mean.

Dataset after Replacing Missing Numerical Values with Mean:
                       Name        Age  Gender Blood Type Medical Condition  \
9995             James Hood  83.000000    Male         A+           Obesity   
9996        Stephanie Evans  47.000000  Female        AB+         Arthritis   
9997   Christopher Martinez  54.000000    Male         B-         Arthritis   
9998            Amanda Duke  84.000000    Male         A+         Arthritis   
9999              Eric King  20.000000    Male         B-         Arthritis   
10000              John Doe  51.449765    Male         A+      Hypertension   
10001            Jane Smith  40.000000  Female        NaN            Cancer   
10002           Alice Brown  35.000000  Female         O-               NaN   
10003             Bob White  51.449765    Male         B+            Asthma   
10004             Eve Black  55.000000     NaN         A-          Diabetes   

      Date of Admis

In [ ]:
# 3. Perform the mode imputation for a categorical data.
for col in df.select_dtypes(include=[object]).columns:
    if missing_percentage[col] < 10:
        df[col].fillna(df[col].mode()[0])
        print(f"Missing values in '{col}' filled with mode: {df[col].mode()[0]}") # Print output for this step
print("\nDataset after Mode Imputation for Categorical Data:")
print(df.tail(10))

Missing values in 'Name' filled with mode: Michael Johnson
Missing values in 'Gender' filled with mode: Female
Missing values in 'Blood Type' filled with mode: AB-
Missing values in 'Medical Condition' filled with mode: Asthma
Missing values in 'Date of Admission' filled with mode: 2019-04-12
Missing values in 'Doctor' filled with mode: Michael Johnson
Missing values in 'Hospital' filled with mode: Smith PLC
Missing values in 'Insurance Provider' filled with mode: Cigna
Missing values in 'Admission Type' filled with mode: Urgent
Missing values in 'Discharge Date' filled with mode: 2021-11-28
Missing values in 'Medication' filled with mode: Penicillin
Missing values in 'Test Results' filled with mode: Abnormal

Dataset after Mode Imputation for Categorical Data:
                       Name        Age  Gender Blood Type Medical Condition  \
9995             James Hood  83.000000    Male         A+           Obesity   
9996        Stephanie Evans  47.000000  Female        AB+         Arth

In [ ]:
# 4. Perform a KNN Imputer to estimate the missing values.
imputer = KNNImputer(n_neighbors=2)
df_imputed = pd.DataFrame(imputer.fit_transform(df.select_dtypes(include=[np.number])), columns=df.select_dtypes(include=[np.number]).columns)
df.update(df_imputed)


In [ ]:
print("Dataset after KNN Imputer for Numerical Data:")
print(df.tail(10))

Dataset after KNN Imputer for Numerical Data:
                       Name        Age  Gender Blood Type Medical Condition  \
9995             James Hood  83.000000    Male         A+           Obesity   
9996        Stephanie Evans  47.000000  Female        AB+         Arthritis   
9997   Christopher Martinez  54.000000    Male         B-         Arthritis   
9998            Amanda Duke  84.000000    Male         A+         Arthritis   
9999              Eric King  20.000000    Male         B-         Arthritis   
10000              John Doe  51.449765    Male         A+      Hypertension   
10001            Jane Smith  40.000000  Female        AB-            Cancer   
10002           Alice Brown  35.000000  Female         O-            Asthma   
10003             Bob White  51.449765    Male         B+            Asthma   
10004             Eve Black  55.000000  Female         A-          Diabetes   

      Date of Admission             Doctor                   Hospital  \
9995       

In [ ]:
# 5. Drop the columns with more than 10% missing values and display the size.
df_dropped_cols = df.dropna(axis=1, thresh=len(df) * 0.9)
print("Dataset Size after Dropping Columns with >10% Missing Values:", df_dropped_cols.shape)


Dataset Size after Dropping Columns with >10% Missing Values: (10005, 15)


In [ ]:
# 6. Drop the rows with outlier Z-score value > 3 and display the size.
from scipy.stats import zscore
z_scores = df.select_dtypes(include=[np.number]).apply(zscore)
df_no_outliers = df[(np.abs(z_scores) < 3).all(axis=1)]
print("Dataset Size after Dropping Rows with Outlier Z-score > 3:", df_no_outliers.shape)


Dataset Size after Dropping Rows with Outlier Z-score > 3: (10005, 15)


In [ ]:
# 7. Drop the duplicate rows based on more than 50% of column having same value.
df_no_duplicates = df.drop_duplicates(subset=df.columns[:int(len(df.columns) * 0.5)])
print("Dataset Size after Dropping Duplicate Rows based on >50% Columns:", df_no_duplicates.shape)


Dataset Size after Dropping Duplicate Rows based on >50% Columns: (10005, 15)


In [ ]:
# 8. Rescale your data using min-max normalization for a numerical feature.
scaler = MinMaxScaler()
df['Billing Amount'] = scaler.fit_transform(df[['Billing Amount']])
print("Billing Amount after Min-Max Normalization:")
print(df[['Billing Amount']].head())

Billing Amount after Min-Max Normalization:
   Billing Amount
0        0.744775
1        0.945060
2        0.732201
3        0.455206
4        0.348728


In [ ]:
# 9. Binarize the data by using binarizer class in python
binarizer = Binarizer(threshold=5000)
df['Billing Amount Binarized'] = binarizer.fit_transform(df[['Billing Amount']])
print("Billing Amount after Binarization:")
print(df[['Billing Amount Binarized']].head())


Billing Amount after Binarization:
   Billing Amount Binarized
0                       0.0
1                       0.0
2                       0.0
3                       0.0
4                       0.0


In [ ]:
# 10. Perform the one-hot encoding for a categorical feature.
encoder = OneHotEncoder(sparse=False)
encoded_columns = encoder.fit_transform(df[['Gender']])
encoded_df = pd.DataFrame(encoded_columns, columns=encoder.get_feature_names_out(['Gender']))
df = df.join(encoded_df)
print("Dataset after One-Hot Encoding 'Gender':")
print(df.head())

Dataset after One-Hot Encoding 'Gender':
                  Name   Age  Gender Blood Type Medical Condition  \
0      Tiffany Ramirez  81.0  Female         O-          Diabetes   
1          Ruben Burns  35.0    Male         O+            Asthma   
2            Chad Byrd  61.0    Male         B-           Obesity   
3    Antonio Frederick  49.0    Male         B-            Asthma   
4  Mrs. Brandy Flowers  51.0    Male         O-         Arthritis   

  Date of Admission          Doctor                   Hospital  \
0        2022-11-17  Patrick Parker           Wallace-Hamilton   
1        2023-06-01   Diane Jackson  Burke, Griffin and Cooper   
2        2019-01-09      Paul Baker                 Walton LLC   
3        2020-05-02  Brian Chandler                 Garcia Ltd   
4        2021-07-09  Dustin Griffin    Jones, Brown and Murray   

  Insurance Provider  Billing Amount  Room Number Admission Type  \
0           Medicare        0.744775          146       Elective   
1   UnitedH

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
